In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime

# Dados do dia 19/05
df = pd.read_csv('data 19-05.csv')

# transforma o índice em datetime
df.index = pd.to_datetime(df['index'], format='%Y-%m-%dT%H:%M:%S.%f')
df.drop(columns=['index'], inplace=True)

df.head()

,p00,p01,p02,p03,p04,p05,p06,p07,p08,p09,p10,p11
index,,,,,,,,,,,,
2023-05-19 14:11:45.149,0,0,0,0,0,0,0,499,0,0,0,0
2023-05-19 14:11:46.149,0,0,0,0,0,0,0,204,0,0,0,0
2023-05-19 14:11:46.649,0,0,0,0,0,0,0,251,0,0,0,0
2023-05-19 14:11:47.149,0,0,0,0,0,0,0,191,0,0,0,0
2023-05-19 14:11:47.649,0,0,0,0,0,0,0,192,0,0,0,0


In [2]:
# Captura poses Vanessa
df_poses_vanessa = df.iloc[14:7448, :]
# px.line(df_poses_vanessa, x=df_poses_vanessa.index, y=df_poses_vanessa.columns)

# Captura poses Bruno
df_poses_bruno = df.iloc[7509:16398, :]

# Dados captura Laguardia
df_poses_laguardia = pd.read_csv('data.csv')

# Captura poses Laguardia
df_poses_laguardia.index = pd.to_datetime(df_poses_laguardia['index'], format='%Y-%m-%dT%H:%M:%S.%f')
df_poses_laguardia.drop(columns=['index'], inplace=True)
df_poses_laguardia = df_poses_laguardia.iloc[8750:, :]

In [3]:
# script de detecção de poses

import pprint  # print bonito pra mostrar o resultado

def seated(data) -> bool:
    """
    Regras para dizer se o usuário está sentado ou não.
    """
    return not(data['p00'] < 10 and data['p01'] < 10 and data['p05'] < 10 and data['p06'] < 10)


def check_poses(df) -> dict:
    """
    Função que recebe um dataframe e retorna um dicionário
    com os índices de início e fim de cada pose.
    """

    poses = {}                      # dicionário de poses
    pose_atual = 1                  # pose atual
    first_seat = None               # primeiro índice em que o usuário estava sentado
    seated_frames_threshold = 10    # número de frames que o usuário deve estar em pé para considerar que ele está sentado

    seated_frames = 0
    for line in df.iterrows():
        index, data = line

        if seated(data):
            # se a pose atual já foi registrada, continua
            if pose_atual in poses:
                continue
            
            # se o usuário estava em pé, define o índice de início da pose atual
            if seated_frames == 0:
                first_seat = index
            seated_frames += 1
            
            # se o número de frames em que o usuário está sentado superar o threshold,
            # considera que ele está sentado e marca o índice de início da pose atual
            if seated_frames > seated_frames_threshold:
                poses[pose_atual] = {}
                poses[pose_atual]['start'] = first_seat
        
        else:  # não está sentado
            seated_frames = 0
            # se o usuário estava sentado, marca o índice de fim da pose atual
            if pose_atual in poses:
                if 'end' not in poses[pose_atual]:
                    poses[pose_atual]['end'] = index
                    pose_atual += 1  # vai para a próxima pose
    
    return poses


# Poses para cada um
poses_vanessa = check_poses(df_poses_vanessa)
poses_bruno = check_poses(df_poses_bruno)
poses_laguardia = check_poses(df_poses_laguardia)

In [4]:
# cria um dataframe de cada pessoa com as poses catalogadas
df_poses_cat_vanessa = df_poses_vanessa.copy()
df_poses_cat_vanessa['pose'] = 0
for pose, indices in poses_vanessa.items():
    df_poses_cat_vanessa.loc[indices['start']:indices['end'], 'pose'] = pose

df_poses_cat_bruno = df_poses_bruno.copy()
df_poses_cat_bruno['pose'] = 0
for pose, indices in poses_bruno.items():
    df_poses_cat_bruno.loc[indices['start']:indices['end'], 'pose'] = pose

df_poses_cat_laguardia = df_poses_laguardia.copy()
df_poses_cat_laguardia['pose'] = 0
for pose, indices in poses_laguardia.items():
    df_poses_cat_laguardia.loc[indices['start']:indices['end'], 'pose'] = pose

# concatena os dataframes
df_poses_cat = pd.concat([df_poses_cat_vanessa, df_poses_cat_bruno, df_poses_cat_laguardia])

In [5]:
# Dados da captura livre de poses Vanessa
df_aux_vanessa = df.copy()

df_aux_vanessa['pose'] = 0

df_aux_vanessa["pose"][19172:19510]= 3

df_aux_vanessa["pose"][19510:20392] = 1

df_aux_vanessa["pose"][20392:21766]= 3

df_aux_vanessa["pose"][21766:21980] = 1

df_aux_vanessa["pose"][21980:22552] = 8

df_aux_vanessa["pose"][22552:22933] = 7

poses_vanessa_livre = df_aux_vanessa.iloc[19172:22933, :]


### Classificando poses

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct, Matern, WhiteKernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# separa os dados de treino e teste

# X = df_poses_cat.drop(columns=['pose'])
# y = df_poses_cat['pose']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=48)


concat = pd.concat([df_poses_cat_vanessa])
X = concat.drop(columns=['pose'])
y = concat['pose']
X_train, y_train = X, y

# df_poses_cat_laguardia_copy = df_poses_cat_laguardia.copy()
# X_test = df_poses_cat_laguardia.drop(columns=['pose'])
# y_test = df_poses_cat_laguardia['pose']



X_train, y_train = X, y
X_test = poses_vanessa_livre.drop(columns=['pose'])
y_test = poses_vanessa_livre['pose']

print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')

X_train: (7434, 12)
X_test: (3761, 12)


In [7]:
# Lista de modelos a serem testados
models = [
    MLPClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

# Lista de parâmetros 
param_grids = [
    # MLPClassifier
    {
        'hidden_layer_sizes': [(10,), (50,), (100,)],
        'activation': ['relu'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive'],
        'max_iter': [300, 500]
    },
    # KNeighborsClassifier
    {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'p': [1, 2, 3]
    },
    # DecisionTreeClassifier
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10]
    },
    # RandomForestClassifier
    {
        'n_estimators': [10, 50, 100],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10]
    },
    # AdaBoostClassifier
    {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.1, 0.5, 1.0],
        'algorithm': ['SAMME', 'SAMME.R']
    },
    # GaussianNB
    {
        'var_smoothing': [1e-9, 1e-8, 1e-7]
    },
    # QuadraticDiscriminantAnalysis
    {
        'reg_param': [0.0, 0.1, 0.5, 1.0]
    }
]

In [8]:
# Cria e treina a grid de busca
for model, param_grid in zip(models, param_grids):
    break
    print(f'Treinando {model.__class__.__name__}')
    grid = GridSearchCV(model, param_grid, scoring='accuracy', refit='accuracy', verbose=1, cv=2)
    grid.fit(X_train, y_train)
    print(f'Melhores parâmetros: {grid.best_params_}')
    print(f'Melhor score: {grid.best_score_}')
    print(f'Score no teste: {grid.score(X_test, y_test)}')
    print('')

In [9]:
# Treina a árvore de decisão com os melhores parâmetros
model_tree = DecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_split=10)
model_tree.fit(X_train, y_train)
print(f'Score no treino: {model_tree.score(X_train, y_train)}')
print(f'Score no teste: {model_tree.score(X_test, y_test)}')

# Mosta a importância de cada feature
print('Importância das features:')
for feature, importance in zip(X_train.columns, model_tree.feature_importances_):
    print(f'{feature}: {importance}')

Score no treino: 0.9923325262308313
Score no teste: 0.2911459718160064
Importância das features:
p00: 0.2533140473788084
p01: 0.1618000671274266
p02: 0.0007380987154607252
p03: 0.06520153569083295
p04: 0.008904652271993383
p05: 0.06606663832484375
p06: 0.1547727579130169
p07: 0.08857958148931773
p08: 0.07358060740402979
p09: 0.05884898803941933
p10: 0.005002541667912017
p11: 0.0631904839769383


In [10]:
# Cria o modelo de Random Forest com os melhores parâmetros
model_rf = RandomForestClassifier(max_depth=None, min_samples_split=5, n_estimators=100)
model_rf.fit(X_train, y_train)

# Mostra o score no treino
print(f'Score no treino: {model_rf.score(X_train, y_train)}')

# Mostra o score no teste
print(f'Score no teste: {model_rf.score(X_test, y_test)}')

# Prediz todos os dados catalogados
# df_poses_cat['pose_pred'] = model_rf.predict(X)

Score no treino: 0.998923863330643
Score no teste: 0.28476469024195694


In [11]:
poses_vanessa_livre['pose_pred'] = model_rf.predict(X_test)

C:\Users\lagua\AppData\Local\Temp\ipykernel_16100\3170793195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poses_vanessa_livre['pose_pred'] = model_rf.predict(X_test)


In [12]:
# Mostra o resultado nos dados catalogados
fig = px.line(poses_vanessa_livre, x=poses_vanessa_livre.index, y=['pose', 'pose_pred'])
fig.show()

# Fazendo o gráfico de poses com relação a p04 e p05
Segundo a análise do notebook anterior.

In [14]:
poses_vanessa_livre['pose'] = poses_vanessa_livre['pose'].astype('category')
px.scatter(poses_vanessa_livre, x='p04', y='p05', color='pose')

C:\Users\lagua\AppData\Local\Temp\ipykernel_16100\2453268749.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
df_poses_cat_vanessa['pose'] = df_poses_cat_vanessa['pose'].astype('category')
px.scatter(df_poses_cat_vanessa, x='p04', y='p05', color='pose')